#### When the data might be stored in more than one vector store, we need to be able to route the query to the appropriate datasource to retrieve relevant docs.

### Logical Routing

We give the LLM knowledge of the various data sources at our disposal and then let the LLM reason which data source to apply based on the user’s query.
In order to achieve this, we make use of function-calling models like GPT-3.5 Turbo to help classify each query into one of the available routes.

In [1]:
from typing import Literal
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

In [10]:
# Data model
class RouteQuery(BaseModel):
    """Route a user's query to the most relevant datasource."""
    datasource: Literal["python_docs", "js_docs"] = Field(
        description="""Given users question, choose which data source is most relevant.""",
    )

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
structured_llm = llm.with_structured_output(RouteQuery)

system = """You are an expert at routing a user question to the relevant data source.
Based on the programming language the question is referring to, route it to the relevant data source.
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

router = prompt | structured_llm

In [12]:
question = """Why doesn't the following code work: 
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages(["human", "speak in {language}"])
prompt.invoke("french")
"""

result = router.invoke({"question" : question})
result, result.datasource

(RouteQuery(datasource='python_docs'), 'python_docs')